In [103]:
#import the packages you need

import requests
import config
import mysql.connector 
import time

In [443]:
#enter the keys that you want into the keys1 list
#this function will remove unneeded keys

def dict_clean_businesses(df):
    keys1 = ['id', 'vote_average', 'title', 'genre_ids', 'release_date', 'actor']
    keys2 = list(df[0].keys())

    for dictionary in df:
        for key in keys2:
            if key not in keys1:
                del dictionary[key]
    return df

In [130]:
#parsing the genre table via api
def genre_id():
    parameters = {'api_key': config.api_key} #these parameters go in to the URL
    response = requests.get('https://api.themoviedb.org/3/genre/movie/list?language=en-US', params = parameters)
    response.status_code #check this, should be 200
    return response.json()

In [463]:
#function to obtain the movie dictionary data "per page"
def api_movies():
    parameters = {'api_key': config.api_key, 'sort_by': 'vote_average.desc', 'vote_count.gte': 100, 'page': 1, 'with_original_language': 'en'}
    #the 'page' key in the parameters will be a variable to be looped over
    response = requests.get('https://api.themoviedb.org/3/discover/movie', params = parameters)
    time.sleep(1)  #pauses code for a second to prevent too many parses
    response.status_code #status code should be 200
    
    return response.json()['results']
# api_movies()
# dict_clean_businesses(webscrape())



In [145]:
from bs4 import BeautifulSoup #Now we begin webscraping

In [414]:

html_page = requests.get('https://www.themoviedb.org/movie/'+'182560'+'/cast')
#182560 is a movie id 'variable' that will be iterated over
soup = BeautifulSoup(html_page.content, 'html.parser')
#standard syntax of webscraping

#1) inspect page and find the actors hightlight
#2) select the nearest tag w/ class = *div* container
div = soup.find('ol', class_='people credits')
all_divs = div.findAll('div', class_='info')
#^find 'div' tag with class=info 
#and work from there using:
#nextSibling, 

nextSibling, previousSibling, children, parent, attrs, find(), findAll()

In [415]:
def movie_actors():
    
    html_page = requests.get('https://www.themoviedb.org/movie/'+'182560'+'/cast')
    soup = BeautifulSoup(html_page.content, 'html.parser')
    div = soup.find('ol', class_='people credits')
    all_divs = div.findAll('div', class_='info')
    
    actors = []
    for tag in all_divs:
        actors.append(tag.text.split('\n')[1])
    return actors
movie_actors()

['Charlize Theron',
 'Nicholas Hoult',
 'Chloë Grace Moretz',
 'Christina Hendricks',
 'Tye Sheridan',
 'Corey Stoll',
 'Andrea Roth',
 'Sterling Jerins',
 'Shannon Kook',
 'Drea de Matteo',
 'Sean Bridgers',
 'J. Larose',
 'Jennifer Pierce Mathus',
 'Denise Williamson',
 'Natalie Precht',
 'Madison McGuire',
 'Addy Miller',
 'Jeff Chase',
 'Laura Cayouette',
 'Richard Gunn',
 'Glenn Morshower',
 'Lori Z. Cordova',
 'Michael Crabtree',
 'John F. Daniel',
 'Steve Shearer',
 'Dan Hewitt Owens',
 'Gillian Flynn',
 'Tyler Forrest',
 'Dora Madison']

In [447]:
#1000 movies 
#actors from the movies

#actor_id below
div = soup.find('ol', class_='people credits') 
all_divs = div.findAll('div', class_='info')
all_divs[0].find('a').attrs['href'].split('/')[2].split('-')[0]
# div

'6885'

In [452]:
#steps taken to webscrape
from bs4 import BeautifulSoup

html_page = requests.get('https://www.themoviedb.org/person/119589')
soup = BeautifulSoup(html_page.content, 'html.parser')

#inspect Donald Glover
#acting
div = soup.find('section', class_='facts left_column')
div.findAll('p')
list(div.findAll('p')[0].children)[0].text
list(div.findAll('p')[0].children)[1].strip()

'Acting'

In [459]:
def actor_info():
    html_page = requests.get('https://www.themoviedb.org/person/' + '119589')
    soup = BeautifulSoup(html_page.content, 'html.parser')
    div = soup.find('section', class_='facts left_column')
    traits = []
    for i in range(len(div.findAll('p'))-4):
        traits.extend(list(div.findAll('p')[i].children)[1].split())
    traits[4:8] = [' '.join(traits[4:8])]
    
#     traits.extend(traits[4].split(','))
#     print(traits)
#     del traits[4:6]
#     del traits[5]
#the above code is if we only want STATES (i.e. Georgia, California) the code is unfinished
    return traits

actor_info()
        
        

['Acting', 'Male', '27', '1983-09-25', 'Stone Mountain, Georgia, USA']

In [458]:
list(div.findAll('p')[0].children)[1].split()[0]

'Acting'

In [ ]:
# all_divs = div.findAll('div', class_='info')

In [470]:
import requests
from bs4 import BeautifulSoup
import mysql.connector
from mysql.connector import errorcode
import datetime
import config

conn = mysql.connector.connect(
                host = "rami-flatiron.ci2poaotvf7v.us-east-2.rds.amazonaws.com",
                user = "admin_rami",
                passwd = "oolP753cD635",
                database = 'movie_insights'
                )
c = conn.cursor()
for i in range(38,52):
    params = {'api_key': config.api_key, 'sort_by':'vote_average.desc', 'vote_count.gte': 100,
              'page': i, 
              'with_original_language': 'en'}
    base_url = config.base_url
    response = requests.get(base_url + '/discover/movie', params = params)
    if response.status_code != 200:
        print(f'Break at {i}, status code: {response.status_code}')
        break
    
    less_info = list(map(lambda x: {'id':x['id'], 
                                    'vote_average': x['vote_average'],
                                    'title': x['title'],
#                                   'genre_ids': x['genre_ids'],
                                   },response.json()['results']))
    

    def scrape_info(movie_dict):
        movie_id = movie_dict['id']
        html_page = requests.get('https://www.themoviedb.org/movie/' + str(movie_id)) #Make a get request to retrieve the page
        soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
        releases = soup.find('ul', class_="releases")
        revenue = releases.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling
        try:
            movie_dict['revenue'] = float(str(revenue.text).replace(revenue.strong.text, "").replace('$','').replace(',','').strip())
        except:
            movie_dict['revenue'] = None
        budget = releases.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling
        try:
            movie_dict['budget'] = float(str(budget.text).replace(budget.strong.text, "").replace('$','').replace(',','').strip())
        except:
            movie_dict['budget'] = None
        release_date = releases.div.previous_sibling.previous_sibling.previous_sibling.replace('\n','').strip()
        movie_dict['release_date'] = datetime.datetime.strptime(release_date, "%B %d, %Y").date()
#         movie_dict['language'] = list(releases.nextSibling.nextSibling.children)[1].strip()
#         html_page = requests.get('https://www.themoviedb.org/movie/'+ str(movie_id) + '/cast')
#         soup = BeautifulSoup(html_page.content, 'html.parser')
#         div = soup.find('ol', class_='people credits')
#         all_divs = div.findAll('div', class_='info')
    
#         actors = []
#         for tag in all_divs:
#             actors.append(tag.text.split('\n')[1])
#         movie_dict['actors'] = actors
        return movie_dict

    movie_info = list(map(scrape_info,less_info))
    
    def populate_db(records, table_name):
#         INSERT_STRING = ("INSERT INTO {table_name} "
#                    "(id, vote_average, title, genre_ids, revenue, budget, release_date) "
#                    "VALUES (%s, %s, %s, %s)")
        INSERT_STR = f'INSERT INTO {table_name} (id, vote_average, title, revenue, budget, release_date) ' 
        VALUE_STR = 'VALUES (%s, %s, %s, %s, %s, %s)'
        INSERT_QUERY = INSERT_STR + VALUE_STR
    
        for rec in movie_info:
            values = tuple(rec.values())
            c.execute(INSERT_QUERY, values)
        conn.commit()

    populate_db(movie_info,'movie_info')
c.close()
conn.close()  